#### Import modules and define parameters


In [10]:
from pathlib import Path

import pandas as pd

from impreproc.images import Image, ImageList
from impreproc.renaming import ImageRenamer

# Define parameters
# Note that in Jupyter Notebooks, the path are relative to the notebooks folder!

# Image path. It can be a single folder containing all images or a folder cointiaing subfolders with the images (recursive option must be set to True)
data_dir = Path("/mnt/phd/ronconi/img_potree")

# Path to the prior classes file (optional). It is a .csv file with two columns: "name" and "class", without the header. If present, a column "class" will be added to the output Pandas Dataframe
# prior_class_file = data_dir / "prior_classes.csv"

# Define extensions of the file to read as a list of strings. 
# It can be a single file extension ["JPG"] or multiples extensions ["jpg", "PNG", "DNG"] 
image_ext = ["jpg"] 

# Read images recursively in subbofolders
recursive = False

# Destination folder
dest_folder = data_dir / "renamed/"

# Base name for the renamed images  
base_name = "DJI"

# Add a progressive ID after the base name
progressive_id = True

# Overlay name on the image
overlay_name = True

# Delete original files
delete_original = False

# Process images in parallel using all the cores available in the machine for speed up the process (it only works when progressive_id is False)
parallel = True

/home/francesco/phd/impreproc/src/impreproc/renaming.py:211: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if dest_folder is "previews":


#### Get list of files


In [3]:
# Get image list
files = ImageList(data_dir, image_ext=image_ext, recursive=recursive)

# See first 5 files to check if everything is ok
files.head

/mnt/phd/ronconi/img_potree/DJI_0009.JPG
/mnt/phd/ronconi/img_potree/DJI_0012.JPG
/mnt/phd/ronconi/img_potree/DJI_0013.JPG
/mnt/phd/ronconi/img_potree/DJI_0020.JPG
/mnt/phd/ronconi/img_potree/DJI_0021.JPG


#### Rename images and saving output to Pandas df

In [3]:
# Create ImageRenamer object
renamer = ImageRenamer(
    image_list=files,
    dest_folder=dest_folder,
    base_name=base_name,
    progressive_ids=progressive_id,
    delete_original=delete_original,
    # parallel=parallel,
    # prior_class_file=prior_class_file,
)

# Rename files and get Pandas Dataframe with old and new names
df = renamer.rename()

# Save Pandas Dataframe as .csv and .parquet file
df.to_csv(dest_folder/"renaming_dict.csv", index=False)
df.to_parquet(dest_folder/"renaming_dict.parquet")

100%|██████████| 25/25 [00:06<00:00,  3.97it/s]


In [10]:
# Save Pandas Dataframe as .csv and .parquet file
df.to_csv(dest_folder/"renaming_dict.csv", index=False)
df.to_parquet(dest_folder/"renaming_dict.parquet")

#### Make previews for Potree

In [5]:
# Make previews for Potree by overlaying the name on the image
# This method is not fully implemented yet (resizing and normalizing by the focal length is missing)

# Create ImageRenamer object
renamer = ImageRenamer(
    image_list=files,
    dest_folder=data_dir / "previews",
    base_name=base_name,
)
renamer.make_previews()

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:12<00:00,  2.07it/s]


In [12]:
# Low light image enhancement
from impreproc.thirdparty.low_light_image_enhancement import enhance_image_exposure
from pathlib import Path
import cv2 

file = "data/DJI_20230516113434_0070.JPG"
# parser.add_argument("-g", '--gamma', default=0.6, type=float,
#                     help="the gamma correction parameter.")
# parser.add_argument("-l", '--lambda_', default=0.15, type=float,
#                     help="the weight for balancing the two terms in the illumination refinement optimization objective.")
# parser.add_argument("-ul", "--lime", action='store_true',
#                     help="Use the LIME method. By default, the DUAL method is used.")
# parser.add_argument("-s", '--sigma', default=3, type=int,
#                     help="Spatial standard deviation for spatial affinity based Gaussian weights.")
# parser.add_argument("-bc", default=1, type=float,
#                     help="parameter for controlling the influence of Mertens's contrast measure.")
# parser.add_argument("-bs", default=1, type=float,
#                     help="parameter for controlling the influence of Mertens's saturation measure.")
# parser.add_argument("-be", default=1, type=float,
#                     help="parameter for controlling the influence of Mertens's well exposedness measure.")
# parser.add_argument("-eps", default=1e-3, type=float,
#                     help="constant to avoid computation instability.")

gamma = 0.6
lambda_ = 0.15
lime = True
sigma = 3
bc = 1
bs = 1
be = 1
eps = 1e-3

fname = Path(file)
root_dir = Path.cwd().parent
image = cv2.imread(str(root_dir/fname))   
enhanced_image = enhance_image_exposure(image, gamma, lambda_, not lime,
                                        sigma=sigma, bc=bc, bs=bs, be=be, eps=eps)
method = "LIME" if lime else "DUAL"
out_name = f"{fname.stem}_{method}_g{gamma}_l{lambda_}{fname.suffix}"
cv2.imwrite(str(fname.parent/out_name), enhanced_image)